In [1]:
from google.colab import drive
drive.mount("/content/grive")

Mounted at /content/grive


#This task is based on the essay which can be found in following link

(https://medium.com/@polanitzer/logistic-regression-in-python-predict-the-probability-of-default-of-an-individual-8a0091da3775)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from numpy import sqrt
from numpy import argmax
from sklearn.datasets import make_classification
from matplotlib import pyplot
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
#Import svm model
from sklearn import svm
from sklearn.metrics import r2_score, explained_variance_score, confusion_matrix, accuracy_score, classification_report, log_loss
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

y — has the loan applicant defaulted on his loan? (binary: “1”, means “Yes”, “0” means “No”)

In [4]:
df=pd.read_excel("/content/grive/MyDrive/Bank credit risk LG/Bank Default.xlsx")

We examine if there are any na values

In [ ]:
df.shape
df.head()

We dont have any missing values

In [ ]:
df.isnull().sum()

### 1.2 see the data type

In [ ]:
df.dtypes

As we can observe from the above table  education is the only variable that is object. We have to turn it in string

In [4]:
df["education"] = df["education"].astype(str)

Investigate all the unique values of the variables

In [ ]:

for column in df:
    try:
        unique_vals = np.unique(df[column])
    except:
        unique_vals = df[column].unique()
        
    nr_values = len(unique_vals)
    if nr_values < 10:
        print('The number of values for feature {} :{} -- {}'.format(column, nr_values,unique_vals))
    else:
        print('The number of values for feature {} :{}'.format(column, nr_values))

#Visuallise the data

In [ ]:
numeric2=['age','years_with_current_employer','years_at_current_address','household_income','debt_to_income_ratio', 'credit_card_debt','other_debt']

In [ ]:
# Example 11 - Investigating the outliers

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(8,5)})

for c in numeric2:
    print(c)
    x = df[c].values
    ax = sns.boxplot(x, color = '#D1EC46')
    print('The meadian is: ', df[c].median())
    plt.title(c)
    plt.show()

In [ ]:
df[df["other_debt"]>8000].head(20)

#As we can see from the boxplots 3 variables have outliers.
1.  House hold income
2.  credit_card_debt
3.  other_debt

In [ ]:
df["years_with_current_employer"]
df["credit_card_debt"]

In [ ]:
# Example 8 - Investigating the distribution 
# Increases the size of sns plots
sns.set(rc={'figure.figsize':(8,5)})

numeric2 = [ 'age','years_with_current_employer','credit_card_debt', 'debt_to_income_ratio', 'household_income']
for n in numeric2:
    x = df[n].values
    sns.distplot(x, color = 'blue');

    # Calculating the mean
    mean = df[n].mean()

    #ploting the mean
    plt.axvline(mean, 0,1, color = 'red')
    plt.title(n)
    plt.show()

In [ ]:
sns.kdeplot( df["household_income"].loc[df["y"] == 0], hue=df["y"], shade=True)
sns.kdeplot( df["household_income"].loc[df["y"] == 1], hue=df["y"], shade=True)

In [ ]:
sns.kdeplot( df["other_debt"].loc[df["y"] == 0], hue=df["y"], shade=True)
sns.kdeplot( df["other_debt"].loc[df["y"] == 1], hue=df["y"], shade=True)

In [ ]:
sns.kdeplot( df["debt_to_income_ratio"].loc[df["y"] == 0], hue=df["y"], shade=True)
sns.kdeplot( df["debt_to_income_ratio"].loc[df["y"] == 1], hue=df["y"], shade=True)

In [ ]:
df.columns

Index(['loan_applicant_id', 'age', 'education', 'years_with_current_employer',
       'years_at_current_address', 'household_income', 'debt_to_income_ratio',
       'credit_card_debt', 'other_debt', 'y'],
      dtype='object')

In [ ]:
#sns.pairplot(df.loc[:,df.columns!="education"], hue = 'y')

In [ ]:
table=pd.crosstab(df.education,df.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True)
plt.title("Stacked Bar Chart of Education vs Default")
plt.xlabel("Education")
plt.ylabel("Proportion of Applicants")
plt.savefig("education_vs_def_stack")

we cannot see a lot of results.

#Data preparation

Since we have imbalanced data we have to use smote algorithm

In [5]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, confusion_matrix, accuracy_score, classification_report, log_loss

All the variable have to be numerical

In [6]:
df["education"]=pd.factorize(df["education"])[0]

In [7]:
X=df.loc[:,(df.columns != "y") & (df.columns !="loan_applicant_id")]
y=df["y"]
X

,age,education,years_with_current_employer,years_at_current_address,household_income,debt_to_income_ratio,credit_card_debt,other_debt
0,44,0,10,20,192,1.211664e+09,1.437731e+09,8.886645e+09
1,34,1,3,18,57,1.426423e+09,5.137880e+09,2.992730e+09
2,45,0,14,24,212,7.285681e+09,1.046031e+09,4.985339e+09
3,33,2,12,5,418,1.138627e+09,3.040189e+09,4.455443e+09
4,20,3,4,19,122,2.841849e+09,1.456045e+09,2.011011e+08
...,...,...,...,...,...,...,...,...
41183,49,1,4,4,114,1.229614e+09,4.868954e+09,9.148643e+09
41184,52,4,13,16,178,2.142134e+09,2.321063e+09,1.491935e+09
41185,42,0,18,15,264,9.821804e+09,1.229046e+09,1.363910e+09
41186,21,0,12,24,56,2.399865e+09,8.108453e+09,5.330789e+09


Smote algorithm to have balanced data

In [8]:
X_train, X_test, Y_train, Y_test=train_test_split(X,y,test_size=0.2,random_state=42)
smote=SMOTE()
smote_X, smote_Y=smote.fit_resample(X_train,Y_train)

In [9]:
smote_X=pd.DataFrame(smote_X,columns=X.columns)
smote_X
smote_Y=pd.DataFrame(data=smote_Y)
#columns=["Y"]
#smote_Y

In [ ]:
#Scale data before we use machine learning algorithm

In [10]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
X_train = scale.fit_transform(smote_X)
Y_train=smote_Y
X_test = scale.transform(X_test)

Feature Importance
Steps of Running Feature Importance

1. Split the data into X & y

2. Run a Tree-based estimators (i.e. decision trees & random forests)

3. Run Feature Importance

*We measure the importance of a feature by calculating the increase in the model’s prediction error after permuting the feature

More info: https://christophm.github.io/interpretable-ml-book/feature-importance.html

In [11]:
from sklearn.tree import DecisionTreeClassifier
# Run a Tree-based estimators (i.e. decision trees & random forests)
dt = DecisionTreeClassifier(random_state=15, criterion  = 'entropy', max_depth = 10)
dt.fit(X_train,Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=15)

In [ ]:
X.columns

In [ ]:
X_columns=['age', 'education', 'years_with_current_employer','years_at_current_address', 'household_income', 'debt_to_income_ratio','credit_card_debt', 'other_debt']

In [12]:
#X_columns
# Calculating FI
for i, column in enumerate(X.columns):
    print('Importance of feature {}:, {:.3f}'.format(column, dt.feature_importances_[i]))
    
    fi = pd.DataFrame({'Variable': [column], 'Feature Importance Score': [dt.feature_importances_[i]]})
    
    try:
        final_fi = pd.concat([final_fi,fi], ignore_index = True)
    except:
        final_fi = fi

Importance of feature age:, 0.051
Importance of feature education:, 0.032
Importance of feature years_with_current_employer:, 0.011
Importance of feature years_at_current_address:, 0.062
Importance of feature household_income:, 0.595
Importance of feature debt_to_income_ratio:, 0.156
Importance of feature credit_card_debt:, 0.071
Importance of feature other_debt:, 0.022


In [16]:
final_fi.sort_values(by="Feature Importance Score",ascending=False)

,Variable,Feature Importance Score
4,household_income,0.594895
5,debt_to_income_ratio,0.155904
6,credit_card_debt,0.071349
3,years_at_current_address,0.062012
0,age,0.050605
1,education,0.032434
7,other_debt,0.021674
2,years_with_current_employer,0.011128


1. 'years_at_current_address', 
2. 'household_income'
3. debt_to_income_ratio
4. 'credit_card_debt'
5. 'age'

the scores simply represent the “importance” of each feature. A higher score means that the specific feature will have a larger effect on the model that is being used to predict a certain variable

we keep feautures with importance above 0.05

In [ ]:
# Ordering the data
#final_fi = final_fi.sort_values('Feature Importance Score', ascending = False).reset_index()            
#final_fi

#10. Training & Running the classification model

So we take only the 5 more significant variables to train our model.

In [ ]:
X_train

In [13]:
from sklearn import svm
#clf=svm.SVC()
clf=SVC()
clf.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [14]:
y_pred=clf.predict(X_test)

In [15]:
print("Accuracy:",metrics.accuracy_score(Y_test,y_pred))
print(classification_report(Y_test,y_pred))

Accuracy: 0.8306627822286963
              precision    recall  f1-score   support

           0       0.94      0.86      0.90      7317
           1       0.35      0.60      0.44       921

    accuracy                           0.83      8238
   macro avg       0.65      0.73      0.67      8238
weighted avg       0.88      0.83      0.85      8238



In [ ]:
#Y_pred_prob=clf.predict_proba(X_test)[:,1]     we had to train our model with probability = true

In [16]:
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
precision_score(Y_test,y_pred)
recall_score(Y_test,y_pred)

0.5993485342019544

In [17]:
precision_score(Y_test,y_pred)

0.34980988593155893

#Model Optimazitation 

Tuning Hyperparameters
1. Kernel: linear, poly, rbf, sigmoid
2. Regularization (C)
3. Gamma:
4. probability=True

In [18]:
#param_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#             'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

In [19]:
grid_search = GridSearchCV(clf, param_grid, scoring='recall')

In [ ]:
grid_result = grid_search.fit(X_train, Y_train)

print(f'Best result: {grid_result.best_score_} for {grid_result.best_params_}')

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(Y_test, y_pred, normalize='true'), annot=True, ax=ax)

After we have found the best model, we apply it to all loan applicants to see the probability of default of each..

In [ ]:
df["PD"] = lr2.predict_proba(X)[:,1]
df[["loan_applicant_id", "PD"]].head(10)

#RFE

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

#logreg=LogisticRegression()

sup_Vector=svm.SVC()

rfe=RFE(sup_Vector)

rfe=rfe.fit(X_train,Y_train.values.ravel())

In [ ]:
rfe.support_
rfe.ranking_

data_X1 = pd.DataFrame({
 "Feature": smote_X.columns,
 "Importance": rfe.ranking_},)
data_X1.sort_values(by=["Importance"])

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, confusion_matrix, accuracy_score, classification_report, log_loss

# running the model
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, Y_train)
y_pred = clf.predict(X_test)
print('Score', clf.score(X_train,Y_train))

pred_proba_t = clf.predict_proba(X_test)
log_loss = log_loss(Y_train, pred_proba_t)
log_loss


print('Log Loss', log_loss)